# Content

1. Load the cohere wikipedia dataset
2. Extract the cohere embeddings for 1000 documents with 768 dimensions
3. Select parameters for the SOM by comparing quantization errors
4. Train the SOM on the wikipedia document embeddings
5. Select the BMUs with at least 1 hit
6. Get query
7. Embed query into the same dimensions (768)
8. Normalize the query vector with the same normalization (var) applied on wikipedia embedding during it's training. (only var normalization has been implemented and available in the SOM library)
9. Select the BMU for the query embedding using cosine similarity search (cosine on exp : (1,768) and (89,768))
10. Get all embeddings allocated to that BMU as candidate context embeddings
11. Get the cosine similarities between the candidate embeddings and the query embedding to select the top k=5 context embeddings. ( exp : (1,768) and (13,768))
12. Get the cosine similarity between all document embeddings and the query embedding to select the top k=5 context embeddings. ( exp : (1,768) and (1000,768))
13. COMPARE 11 and 12 contexts in terms of
*   Accuracy
*   Relavance
*   Speed of retrieval
*   Efficiency of retrieval etc.



In [ ]:
!pip install python-docx PyMuPDF pandas tiktoken langchain cohere

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 47.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.5/259.5 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 70.0 MB/s eta 0:00:00


In [ ]:
!pip install fsspec==2023.9.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.4/173.4 kB 12.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2023.9.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.82 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-nvrtc-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_6

In [ ]:
!pip install -U datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 21.9 MB/s eta 0:00:00
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4


In [ ]:
!pip3 install --upgrade git+https://github.com/GayuniBas2001/SOMPY_sevamoo.git

  Cloning https://github.com/GayuniBas2001/SOMPY_sevamoo.git to /tmp/pip-req-build-9qd9ov9o
  Running command git clone --filter=blob:none --quiet https://github.com/GayuniBas2001/SOMPY_sevamoo.git /tmp/pip-req-build-9qd9ov9o
  Resolved https://github.com/GayuniBas2001/SOMPY_sevamoo.git to commit 420bec153a6600857d2160dd1955b2e0f15f799a
  Preparing metadata (setup.py) ... done
  Created wheel for SOMPY: filename=SOMPY-1.1.1-py3-none-any.whl size=29301 sha256=31a5e052175d321c199d5243921bf9967cd8693e3b74416255c013eafdbccff5
  Stored in directory: /tmp/pip-ephem-wheel-cache-xs7_hkz7/wheels/fc/dd/d6/26375a31b7c69ce2502112a68ead58548f7caa4102897a065d
Successfully built SOMPY


In [ ]:
import os
import io
import re
import base64
import logging
import fitz
import docx
import tiktoken
from langchain.text_splitter import RecursiveCharacterTextSplitter
import numpy as np
import time

In [ ]:
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# 01. Loading Wikipedia cohere embeddings

In [ ]:
from datasets import load_dataset

#Load at max 1000 documents + embeddings
max_docs = 1000
docs_stream = load_dataset(f"Cohere/wikipedia-22-12-simple-embeddings", split="train", streaming=True)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

dataset_infos.json: 0.00B [00:00, ?B/s]

In [ ]:
docs_stream

IterableDataset({
    features: ['id', 'title', 'text', 'url', 'wiki_id', 'views', 'paragraph_id', 'langs', 'emb'],
    num_shards: 4
})

In [ ]:
import torch

docs = []
wik_embeddings = []

for doc in docs_stream:
    docs.append(doc)
    wik_embeddings.append(doc['emb'])
    if len(docs) >= max_docs:
        break

wik_embeddings = torch.tensor(wik_embeddings)

In [ ]:
wik_embeddings.shape

torch.Size([1000, 768])

In [ ]:
import numpy as np

np_embeddings = np.array(wik_embeddings)

# 02. Query Embedding

In [ ]:
import cohere
from getpass import getpass

# Ask the user to enter their Cohere API key
cohere_api_key = getpass("Enter your Cohere API key: ")

co = cohere.ClientV2(api_key=cohere_api_key)

Enter your Cohere API key: ··········


Use cohere API key : KMgAyxtG2SjcOkLRvGJpS0ZIYk34RCEKtjs5AMPp

In [ ]:
model = "multilingual-22-12" # To get 768 dimensions like in the wikipedia embeddings, we must use this model for the query
input_type = "search_query"
output_dimension = wik_embeddings.shape[1]

def embed_prompt(prompt: list,
                 model: str = 'multilingual-22-12',
                 input_type: str = 'search_query',
                 output_dimenion: int = wik_embeddings.shape[1]) -> np.ndarray:

    texts = prompt  # Ensure prompt is a list, as co.embed expects a list of texts

    res = co.embed(
        texts=texts,
        model=model,
        input_type=input_type,
        output_dimension=output_dimension, # Specify the output dimension
        embedding_types=["float"],
    )

    if res.embeddings and res.embeddings.float:
        return np.array(res.embeddings.float) # co.embed returns a list of embeddings, one for each text in the input list. If we provide only one prompt, we take the first embedding
    else:
        raise ValueError("Cohere embed API did not return expected embeddings.")

In [ ]:
questions = [
    "What is the international standard that uses the 24-hour clock format?",
    "Which three parts make up the Tanakh in Judaism?",
    "In which three cities was Gotham City filmed for The Dark Knight Rises?",
    "Against which team did Ronaldo score his famous bicycle-kick goal in the 2018 UEFA Champions League?",
    "Which German state’s court ruled on 5 April 2018 that Puigdemont would not be extradited on charges of rebellion?",
    "What is the voltage range typically used to represent the On state in a logic gate?",
    "Where is Canada's national capital, where the federal government meets?",
    "According to the Copenhagen Interpretation, what state is the cat in before the box is opened?",
    "What gas is identified as the main cause of global warming due to burning fossil fuels?",
    "Which Indian leader used peaceful tactics including \"ahimsa\" to lead the freedom movement against British rule?"
]

In [ ]:
original_query_embeddings = embed_prompt(questions)
original_query_embeddings.shape

(10, 768)

In [ ]:
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# 03. Context Retreival from Reduced Dimensions

In [ ]:
from math import sqrt
from sompy.sompy import SOMFactory, SOM
import itertools
from typing import Any, Dict, List, Tuple
from sklearn.metrics.pairwise import cosine_similarity

#cal_quantization_error function is not used in the code. This is to understand how SOM is calculating the qe
def cal_quantization_error(sm: SOM, data: np.ndarray) -> float:
    bmus, squared_partial = sm.find_bmu(data)
    fixed_euclidean_x2 = np.einsum('ij,ij->i', data, data)
    distances = np.sqrt(squared_partial + fixed_euclidean_x2)
    return np.mean(distances)

def train_som_model(np_embeddings, mapsizes, normalizations, initializations, lattices, neighborhoods, trainings, names, rough_len, finetune_len):
    som_fac = SOMFactory()
    sm = som_fac.build(np_embeddings, mapsize=mapsizes, normalization=normalizations, initialization=initializations, lattice=lattices, neighborhood=neighborhoods, training=trainings, name=names)
    sm.train(n_job=1, verbose=False, train_rough_len=rough_len, train_finetune_len=finetune_len)
    topographic_error = sm.calculate_topographic_error()
    quantization_error = sm.quant_error_history[-1]
    logger.info(f"Topographic error = {topographic_error}; Quantization error = {quantization_error}")
    return sm

def find_closest_hit_per_bmu(sm):
    chunk_bmu_indices = sm._bmu[0].astype(int)
    chunk_bmu_qe = sm._bmu[1] #qe between the chunk and it's bmu
    data = sm._data #the chunk embeddings (already normalized)

    closest_indices = np.full(sm.codebook.nnodes, np.nan)
    min_errors = np.full(sm.codebook.nnodes, np.inf)

    for idx, (bmu, err) in enumerate(zip(chunk_bmu_indices, chunk_bmu_qe)):
        if err < min_errors[bmu]:
            min_errors[bmu] = err
            closest_indices[bmu] = idx

    bmu_first_hit_vectors = [data[int(i)] if not np.isnan(i) else None for i in closest_indices]
    return closest_indices, bmu_first_hit_vectors

def cosine_bmu_retrieve(normed_query_embedding, som_model, bmu_hit_vectors_with_indices, top_k: int = 5): #Here we take 5 best matiching BMUs (units with at least 1 hit) for a query
    filtered = [(i, v) for i, v in bmu_hit_vectors_with_indices if v is not None and not np.isnan(v).any()]
    if len(filtered) == 0:
        raise ValueError("No valid BMU vectors available for similarity comparison.")

    indices, vectors = zip(*filtered)
    vectors = np.array(vectors)
    indices = np.array(indices)
    print('number of hit vectors: ',len(indices))

    similarities = cosine_similarity(normed_query_embedding, vectors)
    sorted_indices_desc = np.argsort(similarities[0])[::-1]
    top_k_indices = sorted_indices_desc[:top_k] #Indices from the similarity array
    print(f"top_k_indices : {top_k_indices}")
    selected_bmu_index = indices[top_k_indices] #Indices of the actual BMUs
    print(f"selected_bmu_index : {selected_bmu_index}")
    return selected_bmu_index

def cosine_context_similarity(normed_query_embedding, candidate_vectors_with_indices, top_k: int = 5):
    indices, vectors = zip(*candidate_vectors_with_indices)
    vectors = np.array(vectors)
    logger.info(f"vectors shape : {vectors.shape}") #(cand_count,768)
    indices = np.array(indices)
    logger.info(f"indices shape : {indices.shape}") #(cand_count,)
    logger.info(f"normed_query_embedding shape : {normed_query_embedding.shape}") #(1,768)
    similarities = cosine_similarity(normed_query_embedding, vectors)
    logger.info(f"similarities shape : {similarities.shape}") #(1, valid)
    return similarities

def normalize_reshape (query_embedding, sm):
    vec = query_embedding.reshape(1, -1)
    normed_query_embedding = sm._normalizer.normalize_by(sm.data_raw, vec)
    logger.info(f"new shape : ",vec.shape)
    return normed_query_embedding

def get_som_context(
    query_embedding: np.ndarray,
    som_model,
    bmu_hit_vectors_with_indices,
    chunks,
    rerank_method: str = 'cosine',
    top_k: int = 5,
    bmu_k: int = 5
):
  q_vec = normalize_reshape(query_embedding, som_model)
  logger.info(f"q_vec shape : ",q_vec.shape)
  q_bmus = cosine_bmu_retrieve(q_vec, som_model, bmu_hit_vectors_with_indices, bmu_k)
  logger.info(f"q_bmus to use : ",q_bmus)

  chunk_bmu_indices = som_model._bmu[0].astype(int)

  q_bmus_set = set(q_bmus)
  candidates_emb = [(i, chunks[i]['emb']) for i, bmu_index in enumerate(chunk_bmu_indices) if bmu_index in q_bmus_set]

  logger.info(f"Found {len(candidates_emb)} candidates")

  # d) Rerank based on the specified method
  if rerank_method == 'cosine':
      scores = cosine_context_similarity(q_vec, candidates_emb, top_k)[0]
      sorted_indices_desc = np.argsort(scores)[::-1] # indices of 20 candidates, not the actual indices

      top_k_chunk_indices = [candidates_emb[i][0] for i in sorted_indices_desc[:top_k]]

      logger.info('top_k_chunk_indices : ', top_k_chunk_indices)

      top_k_indices = top_k_chunk_indices

      context = [chunks[i]['text'] for i in top_k_indices]
      context_scores = scores[sorted_indices_desc[:top_k]]

      return context, context_scores

  else:
      raise ValueError(f"Unknown rerank_method: {rerank_method}. Choose 'cosine' or 'quantization_error'.")

In [ ]:
results = [{'mapsize': (10, 10), 'normalization': 'var', 'initialization': 'random', 'lattice': 'hexa', 'neighborhood': 'gaussian', 'training': 'batch', 'name': 'som1', 'rough_len': 800, 'finetune_len': 100, 'test_index': 0, 'topographic_error': np.float64(0.092), 'quantization_error': np.float64(24.17343805953296)}, {'mapsize': (10, 10), 'normalization': 'var', 'initialization': 'random', 'lattice': 'hexa', 'neighborhood': 'gaussian', 'training': 'seq', 'name': 'som1', 'rough_len': 800, 'finetune_len': 100, 'test_index': 1, 'topographic_error': np.float64(0.147), 'quantization_error': np.float64(23.95357177963876)}, {'mapsize': (10, 10), 'normalization': 'var', 'initialization': 'random', 'lattice': 'rect', 'neighborhood': 'gaussian', 'training': 'batch', 'name': 'som1', 'rough_len': 800, 'finetune_len': 100, 'test_index': 2, 'topographic_error': np.float64(0.399), 'quantization_error': np.float64(23.053306262876312)}, {'mapsize': (10, 10), 'normalization': 'var', 'initialization': 'random', 'lattice': 'rect', 'neighborhood': 'gaussian', 'training': 'seq', 'name': 'som1', 'rough_len': 800, 'finetune_len': 100, 'test_index': 3, 'topographic_error': np.float64(0.423), 'quantization_error': np.float64(23.07128526327146)}, {'mapsize': (10, 10), 'normalization': 'var', 'initialization': 'pca', 'lattice': 'hexa', 'neighborhood': 'gaussian', 'training': 'batch', 'name': 'som1', 'rough_len': 800, 'finetune_len': 100, 'test_index': 4, 'topographic_error': np.float64(0.119), 'quantization_error': np.float64(23.89813955987797)}, {'mapsize': (10, 10), 'normalization': 'var', 'initialization': 'pca', 'lattice': 'hexa', 'neighborhood': 'gaussian', 'training': 'seq', 'name': 'som1', 'rough_len': 800, 'finetune_len': 100, 'test_index': 5, 'topographic_error': np.float64(0.119), 'quantization_error': np.float64(23.89813955987797)}, {'mapsize': (10, 10), 'normalization': 'var', 'initialization': 'pca', 'lattice': 'rect', 'neighborhood': 'gaussian', 'training': 'batch', 'name': 'som1', 'rough_len': 800, 'finetune_len': 100, 'test_index': 6, 'topographic_error': np.float64(0.404), 'quantization_error': np.float64(23.062587304824785)}, {'mapsize': (10, 10), 'normalization': 'var', 'initialization': 'pca', 'lattice': 'rect', 'neighborhood': 'gaussian', 'training': 'seq', 'name': 'som1', 'rough_len': 800, 'finetune_len': 100, 'test_index': 7, 'topographic_error': np.float64(0.404), 'quantization_error': np.float64(23.062587304824785)}]

In [ ]:
test_index = 2
test_case = results[test_index]

def reduced_dim_retreival(
    np_embeddings,
    original_query_embeddings,
    test_case,
    sm
):
  som_contexts_scores = []
  map_size = sm.codebook.nnodes

  bmu_first_hit_indices, bmu_first_hit_vectors = find_closest_hit_per_bmu(sm) #index of the first hit chunk and it's vector

  bmu_hit_vectors_with_indices = [(i, v) for i,v in enumerate(bmu_first_hit_vectors)] #idex of the bmu (to represent the cluster) and the vector of the chunk

  for q in original_query_embeddings:
    print('q shape', q.shape)
    som_context, som_context_score = get_som_context(q, sm, bmu_hit_vectors_with_indices, docs)
    som_contexts_scores.append((som_context, som_context_score))

  return som_contexts_scores

In [ ]:
sm = train_som_model(
        np_embeddings,
        test_case['mapsize'],
        test_case['normalization'],
        test_case['initialization'],
        test_case['lattice'],
        test_case['neighborhood'],
        test_case['training'],
        test_case['name'],
        test_case['rough_len'],
        test_case['finetune_len']
  )

In [ ]:
som_contexts_scores = reduced_dim_retreival(np_embeddings, original_query_embeddings, test_case, sm)

number of hit vectors:  88
top_k_indices : [86 47 12  1 55]
selected_bmu_index : [98 52 12  1 61]
number of hit vectors:  88
top_k_indices : [11 68  3 67 38]
selected_bmu_index : [11 76  3 74 42]
number of hit vectors:  88
top_k_indices : [54 55 86 73 67]
selected_bmu_index : [60 61 98 82 74]
number of hit vectors:  88
top_k_indices : [79 72 54 80 64]
selected_bmu_index : [90 80 60 91 70]
number of hit vectors:  88
top_k_indices : [54 72 63 44 86]
selected_bmu_index : [60 80 69 49 98]
number of hit vectors:  88
top_k_indices : [10 47 19  1 56]
selected_bmu_index : [10 52 20  1 62]
number of hit vectors:  88
top_k_indices : [34 33 44 25 86]
selected_bmu_index : [38 37 49 27 98]
number of hit vectors:  88
top_k_indices : [54 26 55 86 10]
selected_bmu_index : [60 28 61 98 10]
number of hit vectors:  88
top_k_indices : [15 86 16  6 55]
selected_bmu_index : [16 98 17  6 61]
number of hit vectors:  88
top_k_indices : [74 64 73 59 44]
selected_bmu_index : [83 70 82 65 49]


In [ ]:
for r in som_contexts_scores:
  # print(f"[{r[1]:.3f}]\t{r[0]}")
  for txt, score in zip(r[0],r[1]):
    print(f"[{score:.3f}]\t{txt}")
  print('\n\n')

[0.100]	"Within one hour, one of the atoms of the radioactive material might decay (or break down because the material is not stable), or it may not. "
[0.087]	The question now is: at the end of the hour, is the cat alive or dead? Schrödinger says that according to the Copenhagen Interpretation, as long as the door is closed, the cat is dead and alive. There is no way to know until the door is opened. But by opening the door, the person is interfering with the experiment. The person and the experiment have to be described with reference to each other.
[0.081]	"A Geiger counter which counts the amount of radioactive decay and a little bit of a radioactive element are in the room. "
[0.046]	Magnotta was previously sought by animal rights groups for allegedly uploading videos of himself killing kittens. There was no accusation for these videos.
[0.045]	Traffic lights change their colors in the same order every time. In most English-speaking countries, traffic lights usually change in this

# 04. Context Retreival from Original Embeddings

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def get_cosine_context(query_embeddings: np.ndarray, chunks, top_k: int = 5):
    vectors = np.array([chunk['emb'] for chunk in chunks])
    texts = [chunk['text'] for chunk in chunks]

    cosine_contexts_scores = []
    similarities = cosine_similarity(query_embeddings, vectors)  # shape: (n_queries, n_chunks)

    for query_sim in similarities:  # shape: (n_chunks,)
        # Get top_k efficiently
        top_k_indices = np.argpartition(-query_sim, top_k)[:top_k]
        top_k_indices = top_k_indices[np.argsort(-query_sim[top_k_indices])]

        context = [texts[i] for i in top_k_indices]
        context_score = [query_sim[i] for i in top_k_indices]
        cosine_contexts_scores.append((context, context_score))

    return cosine_contexts_scores

In [ ]:
cosine_contexts_scores = get_cosine_context(original_query_embeddings, docs)

In [ ]:
for r in cosine_contexts_scores:
  # print(f"[{r[1]:.3f}]\t{r[0]}")
  for txt, score in zip(r[0],r[1]):
    print(f"[{score:.3f}]\t{txt}")
  print('\n\n')

[0.911]	24-hour clock time is used in computers, military, public safety, and transport. In many Asian, European and Latin American countries people use it to write the time. Many European people use it in speaking.
[0.898]	The 24-hour clock is a way of telling the time in which the day runs from midnight to midnight and is divided into 24 hours, numbered from 0 to 23. It does not use a.m. or p.m. This system is also referred to (only in the US and the English speaking parts of Canada) as military time or (only in the United Kingdom and now very rarely) as continental time. In some parts of the world, it is called railway time. Also, the international standard notation of time (ISO 8601) is based on this format.
[0.894]	However, the US military prefers not to say 24:00 - they do not like to have two names for the same thing, so they always say "23:59", which is one minute before midnight.
[0.879]	A time in the 24-hour clock is written in the form hours:minutes (for example, 01:23), or 

In [ ]:
import time
import tracemalloc
import logging

# Enable logging if desired
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

def benchmark_function(func, *args, **kwargs):
    logger.info(f"Running {func.__name__}...")

    # Start memory tracking
    tracemalloc.start()

    # Start timing
    start_time = time.perf_counter()
    result = func(*args, **kwargs)
    end_time = time.perf_counter()

    # Measure memory usage
    current, peak = tracemalloc.get_traced_memory()
    tracemalloc.stop()

    logger.info(f"{func.__name__} completed.")
    logger.info(f"Time taken: {end_time - start_time:.4f} seconds")
    logger.info(f"Peak memory usage: {peak / 1024:.2f} KB")

    return result, end_time - start_time, peak

In [ ]:
# Run and benchmark reduced_dim_retreival
som_contexts_scores, time1, mem1 = benchmark_function(
    reduced_dim_retreival, np_embeddings, original_query_embeddings, test_case, sm
)

# Run and benchmark get_cosine_context
cosine_contexts_scores, time2, mem2 = benchmark_function(
    get_cosine_context, original_query_embeddings, docs
)

# Print final comparison
print("\n🔍 Performance Comparison:")
print(f"reduced_dim_retreival: {time1:.4f} sec, {mem1 / 1024:.2f} KB peak memory")
print(f"get_cosine_context:     {time2:.4f} sec, {mem2 / 1024:.2f} KB peak memory")


🔍 Performance Comparison:
reduced_dim_retreival: 0.1329 sec, 3069.48 KB peak memory
get_cosine_context:     0.0694 sec, 12150.05 KB peak memory


In [ ]:
# Benchmark SOM training
sm, train_time, train_mem = benchmark_function(
    train_som_model,
    np_embeddings,
    test_case['mapsize'],
    test_case['normalization'],
    test_case['initialization'],
    test_case['lattice'],
    test_case['neighborhood'],
    test_case['training'],
    test_case['name'],
    test_case['rough_len'],
    test_case['finetune_len']
)

print("\n🧠 SOM Training Benchmark:")
print(f"train_som_model: {train_time:.4f} sec, {train_mem / 1024:.2f} KB peak memory")


🧠 SOM Training Benchmark:
train_som_model: 179.4612 sec, 30930.50 KB peak memory


In [ ]:
questions_file='Wikipedia_Questions2.txt'

In [ ]:
def get_questions_from_file(questions_file):
  """
  Reads a file and extracts lines that start with a digit as questions.

  Args:
    questions_file: The path to the file containing the questions.

  Returns:
    A list of strings, where each string is a question.
  """
  questions = []

  try:
    with open(questions_file, "r", encoding="utf-8") as file:
      for line in file:
        stripped = line.strip()
        if stripped and stripped[0].isdigit():
          questions.append(stripped)
  except FileNotFoundError:
    print(f"Error: The file '{questions_file}' was not found.")
  except Exception as e:
    print(f"An error occurred while reading the file: {e}")

  return questions

In [ ]:
questions = get_questions_from_file(questions_file)[:10]

for q in questions:
    print(q)

1. What is the international standard that uses the 24-hour clock format?
2. Which three parts make up the Tanakh in Judaism?
3. In which three cities was Gotham City filmed for The Dark Knight Rises?
4. Against which team did Ronaldo score his famous bicycle-kick goal in the 2018 UEFA Champions League?
5. Which German state’s court ruled on 5 April 2018 that Puigdemont would not be extradited on charges of rebellion?
6. What is the voltage range typically used to represent the On state in a logic gate?
7. Where is Canada's national capital, where the federal government meets?
8. According to the Copenhagen Interpretation, what state is the cat in before the box is opened?
9. What gas is identified as the main cause of global warming due to burning fossil fuels?
10. Which Indian leader used peaceful tactics including "ahimsa" to lead the freedom movement against British rule?


In [ ]:
questions = ['1. What is the international standard that uses the 24-hour clock format?',
 '2. Which three parts make up the Tanakh in Judaism?',
 '3. In which three cities was Gotham City filmed for The Dark Knight Rises?',
 '4. Against which team did Ronaldo score his famous bicycle-kick goal in the 2018 UEFA Champions League?',
 '5. Which German state’s court ruled on 5 April 2018 that Puigdemont would not be extradited on charges of rebellion?',
 '6. What is the voltage range typically used to represent the On state in a logic gate?',
 "7. Where is Canada's national capital, where the federal government meets?",
 '8. According to the Copenhagen Interpretation, what state is the cat in before the box is opened?',
 '9. What gas is identified as the main cause of global warming due to burning fossil fuels?',
 '10. Which Indian leader used peaceful tactics including "ahimsa" to lead the freedom movement against British rule?']

### 7.2 Get context for each question

In [ ]:
def embed_all_questions(questions: list[str],
                 model: str = 'multilingual-22-12',
                 input_type: str = 'search_query',
                 output_dimenion: int = wik_embeddings.shape[1]) -> np.ndarray:
  query_embeddings = []
  for q in questions:
    query_embedding = embed_prompt(q)
    query_embeddings.append(query_embedding)
  return query_embeddings

# Function to retrieve SOM context
def get_som_all(questions, query_embeddings):
  all_som_context = []
  for q,q_emb in zip(questions,query_embeddings):
      context,score = get_som_context(q_emb, sm, valid_bmu_hit_vectors_with_indices, docs)
      all_som_context.append(context)
  return all_som_context

# Function to retrieve cosine similarity context
def get_cosine_all(questions, query_embeddings):
  all_cosine_context = []
  for q, q_emb in zip(questions,query_embeddings):
      context,score = get_cosine_context(q_emb, docs)
      all_cosine_context.append(context)
  return all_cosine_context

In [ ]:
query_embeddings = embed_all_questions(questions[:10])
all_som_context = get_som_all(questions[:10], query_embeddings)
print(f'som_context[0] length : {len(som_context[0])}')
all_cosine_context = get_cosine_all(questions[:10], query_embeddings)
print(f'cosine_context[0] length : {len(cosine_context[0])}')

new shape :  (1, 768)
q_vec shape :  (1, 768)
vectors shape : (84, 768)
indices shape : (84,)
normed_query_embedding shape : (1, 768)
similarities shape : (1, 84)
max_similarity_idx : 10
selected_bmu_index : 11
Found 1 candidates
vectors shape : (1, 768)
indices shape : (1,)
normed_query_embedding shape : (1, 768)
similarities shape : (1, 1)
scores length:  1
sorted_indices_desc length:  1
[0]
top_k_chunk_indices :  [936]
new shape :  (1, 768)
q_vec shape :  (1, 768)
vectors shape : (84, 768)
indices shape : (84,)
normed_query_embedding shape : (1, 768)
similarities shape : (1, 84)
max_similarity_idx : 46
selected_bmu_index : 55
Found 9 candidates
vectors shape : (9, 768)
indices shape : (9,)
normed_query_embedding shape : (1, 768)
similarities shape : (1, 9)
scores length:  9
sorted_indices_desc length:  9
[2 0 3 6 1 4 5 8 7]
top_k_chunk_indices :  [7, 5, 8, 11, 6]
new shape :  (1, 768)
q_vec shape :  (1, 768)
vectors shape : (84, 768)
indices shape : (84,)
normed_query_embedding shap

In [ ]:
print(len(all_som_context))
print(len(all_cosine_context))

10
10


In [ ]:
import pandas as pd
from google.colab import files

def create_context_dataframe(questions, all_som_contexts, all_cosine_contexts):
  df = pd.DataFrame({
      "question": questions,
      "som_context": all_som_contexts,
      "cosine_context": all_cosine_contexts,
  })
  return df

def save_and_download_context(df, filename="wikipedia_context_comparison.csv"):
  df.to_csv(filename, index=False)
  files.download(filename)

# ! Download results to local storage

In [ ]:
df = create_context_dataframe(questions, all_som_context, all_cosine_context)
save_and_download_context(df, "wikipedia_context_comparison.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>